# Part 3: Building a web map of Earthquake Events

This is the final part of our 3-part series on Retrieving, Analyzing and Visualizing georeferenced data of earthquakes. We are using the database created in part 2 that contains location data (longitude and latitude), depth and magnitude of earthquake events that occurred near [Fiji](https://es.wikipedia.org/wiki/Fiyi) since 1964. From that database, we will extract some data needed to build a web map of the earthquake events through the [Folium](https://python-visualization.github.io/folium/) Map library. This web map will contain a baselayer, earthquake event locations, and two heatmaps: magnitude and depth of the earthquakes.


We will import all the required Python libraries.

In [1]:
import sqlite3
import numpy as np
import folium
import folium.plugins as plugins
from folium.plugins import MarkerCluster

## Reading the database

We are going to read our database: `earthquakes.sqlite` and extract data about latitude, longitude, magnitude, and depth of the earthquake events, this data will be stored in list objects. To do that we will use a SQL statement embedded in the Python code. Python has its native builtin modules to connect Python program with a database.

In [2]:
# read dataset
conn = sqlite3.connect('eartquakes.sqlite')
cur = conn.cursor()

# create a list of latitude, longitude, mag and depth
lats=[]
lons =[]
mags =[]
depths =[]
cur.execute("SELECT lat, lon, mag, depth FROM quakes")
for row in cur:
    lat = row[0]
    lon = row[1]
    mag = row[2]
    depth = row[3]
    lats.append(lat)
    lons.append(lon)
    mags.append(mag)
    depths.append(depth)
cur.close()

# get a list of latitude and longitude values
locations = list(zip(lats, lons))

# get a list of  magnitude and depth values
magdepth = list(zip(mags, depths))

## Building the web map

We will be using the [Folium](https://python-visualization.github.io/folium/) Map library to set up our web map of earthquakes events. First, we will configure a [popup window](https://www.webopedia.com/TERM/P/pop_up_window.html) to display attributes about magnitude, and depth of each earthquake. Next, we will establish [Cartodb Positron](https://carto.com/blog/getting-to-know-positron-and-dark-matter/) as a [basemap](http://maps.macog.com/MACOG/Help/Content/Client_APIs/SV_User/SVU_AboutBaseMaps.htm). We will use the marker clustering technique to produce an animated point clustering visualization of the earthquake events.  Finally, we will create a heatmap of earthquake magnitude and a heatmap of earthquake depth. More information about the heatmap technique [here](https://en.wikipedia.org/wiki/Heat_map).

In [3]:
# configure map popup
popups = ['magnitude:{}<br>depth:{}'.format(mags, depths) for (mags, depths) in magdepth]

# create basemap
map = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=5
)

# create marker clusters
marker_cluster = MarkerCluster(
    locations=locations,
    popups=popups,
    name='Earthquake Locations',
    overlay=True,
    control=True,

)
marker_cluster.add_to(map)

# create heatmap of earthquake magnitude
data_heat_mag = list(zip(lats, lons, mags))
mag_hm = plugins.HeatMap(data_heat_mag,
                         name='Heatmap of Earthquake Magnitudes',
                         overlay=True,
                         control=True,
                         show=False
                        )
mag_hm.add_to(map)

# create heatmap of earthquake depth
data_heat_depth = list(zip(lats, lons,depths))
dep_hm = plugins.HeatMap(data_heat_depth,
                         name='Heatmap of Earthquake Depths',
                         overlay=True,
                         control=True,
                         show=False
                        )
dep_hm.add_to(map)

# add map control layer
folium.LayerControl(collapsed=False).add_to(map)

## Displaying the map

The map will be displayed directly on this notebook and saved as an HTML file.

In [5]:
# dislay the map
map

In [6]:
# sav the map
map.save('map.html')

#### Open 'map.html' in a browser to visualize: Earthquake Locations, Magnitude Heatmap and Depth Heatmap